In [1]:
%matplotlib notebook
import os
from adm_library import *

## 0 Parameter definitions

### 0.1 Ingesting the [X,Y,Z,Rx,Ry] values from Tyler's spreadsheet

In [2]:
fname = 'files/SES-cold3-C4- 5DoF inputs_template.xlsx'
sheetname = 'Input for Tyler analysis'
#fname = 'SES-cold plateau 3 - circuit 2.xlsx'
#sheetname = 'Input for Tyler analysis'

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=1,usecols='B:N')

focal_length = spreadsheet.iloc[0][0]-110 # focal length is track length -110mm, per TG
sMPA_to_WCS = spreadsheet.iloc[3,:3].astype(float)
sMPA_angle_to_WCS_deg = spreadsheet.iloc[6,:3].astype(float)
GSA_angle_WCS_deg = spreadsheet.iloc[7][0]
sMask=np.array(spreadsheet.iloc[10,1:4]).astype(float)

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=15,usecols='B:N')
pose_actual = spreadsheet.iloc[0:9,0:6]
pose_actual = pose_actual.set_index('Name').sort_index()
pose_actual.columns = ['X','Y','Z','Rx','Ry']
X_5DOF,Y_5DOF,Z_5DOF,rx_deg_5DOF,ry_deg_5DOF = [[*pose_actual['X'].values.astype(float)],[*pose_actual['Y'].values.astype(float)],[*pose_actual['Z'].values.astype(float)],
                                                [*pose_actual['Rx'].values.astype(float)],[*pose_actual['Ry'].values.astype(float)]]
pose_encoders = spreadsheet.iloc[0:9,7:13]
pose_encoders = pose_encoders.set_index('Name.1').sort_index()

print('Provided 5DOF position in real space: \n',pose_actual,'\n')
print('Provided 5DOF encoder values: \n',pose_encoders)


Provided 5DOF position in real space: 
                X          Y          Z        Rx         Ry
Name                                                       
PDI1 -109.089755  -4.477723  21.386613 -0.941514  10.804298
PDI2  111.429511  -3.828453  21.664246 -0.884323 -10.784784
PDI3   39.795278  52.011229  11.044961  4.741583  -3.798262
PDI4  -38.810833  51.903508  11.309534  4.728098   3.961031
PR1     0.943594   1.280695  -0.338386  0.031453   0.035137
PR2  -109.505749 -45.867074  22.370475 -4.911732  10.680266
PR3   111.309055 -45.373337  22.058458 -4.857077 -10.600321
PR4   103.076801  59.765562  30.696044  5.307985 -10.311998
PR5   -101.43058   59.40225  30.809096  5.269783  10.410043 

Provided 5DOF encoder values: 
                  X          Y          Z        Rx         Ry
Name.1                                                       
PDI1   -109.062609  -4.434582  21.454053 -0.933014  10.805269
PDI2    111.310127  -3.744227  21.815125 -0.889186 -10.782564
PDI3     39.730784

### 0.1.1 If both the pose_actual and pose_encoder arrays are provided in the spreadsheet above, convert between the two to make sure they both agree. 

In [3]:
#No point doing the following if one or the other array is completely empty
if (not any(np.isfinite(pose_encoders.values.astype(float)).ravel())) and (not any(np.isfinite(pose_actual.values.astype(float)).ravel())):

    print('(Calculated 5DOF position) - (Actual 5DOF position (from the spreadsheet)): ')
    calculated_pose_actual = pd.DataFrame(columns=pose_actual.columns)
    for index in [val for val in pose_actual.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_actual = pd.concat((calculated_pose_actual,calculate_5DOF_from_encoders(pd.DataFrame(pose_encoders.loc[index]).T)))
    print(calculated_pose_actual-pose_actual,'\n\n')
    
    print('(Calculated 5DOF encoders) - (Actual 5DOF encoders (from the spreadsheet)): ')
    calculated_pose_encoders = pd.DataFrame(columns=pose_encoders.columns)
    for index in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_encoders = pd.concat((calculated_pose_encoders,calculate_encoders_from_5DOF(pd.DataFrame(pose_actual.loc[index]).T)))
    print(calculated_pose_encoders-pose_encoders)

### 0.2 If the "pose_actual" section of the supplied spreadsheet was empty, build a new "pose_actual" array based on the input "pose_encoder" array.

In [4]:
if not any(np.isfinite(pose_actual.values.astype(float)).ravel()):
    pose_actual = convert_pose_encoders_to_pose_actual(pose_encoders)
    print('NEWLY calculated 5DOF position in real space based on recorded 5DOF encoder values: \n',pose_actual)

# 1. Updating Poses with ADM Data

## 1.1 ADM Baseline  

#### 1.1.1 Input your baseline ADM values

In [5]:
#p_null_PATB_baseline_encoder = pd.DataFrame(dict({'X':88.435, 'Y':109.266,'Z':0,
#                                                  'Rx':0.1018,'Ry':0.0856,
#                                                  'z_sMATF':4.027174*1000.,
#                                                  'z_PATB':3.455684*1000.,
#                                                  'z_type':'retro',
#                                                  'sMATF AC AZ':0.0041,
#                                                  'sMATF AC EL':0.006,
#                                                  'PATB AC AZ':0.0041,
#                                                  'PATB AC EL':0.0035,
#                                                  'sMATF Pri LED X':811.9,
#                                                  'sMATF Pri LED Y':604.6,
#                                                  'PATB Pri LED X':812.6,
#                                                  'PATB Pri LED Y':605.9,
#                                                  'date':'4/1/2023'
#                                                 }),index=['PATB'])
p_null_PATB_baseline_encoder = get_data_from_ADM_log('SES CP2','retro',index_name='PATB',print_details=True) 
p_null_PATB_baseline = calculate_5DOF_from_encoders(p_null_PATB_baseline_encoder)

p_null_offset = np.array([-90.,-110.,-10.]) #PATB is not on the POA boresight. It is offset by this much
p_null_PATB_baseline_encoder


X                               88.435
Y                              109.266
Z                                    0
Rx                              0.1018
Ry                              0.0856
z_sMATF                       4027.174
z_PATB                        3455.684
z_type                           retro
sMATF AC AZ                     0.0041
sMATF AC EL                      0.006
PATB AC AZ                      0.0041
PATB AC EL                      0.0035
sMATF Pri LED X                  811.9
sMATF Pri LED Y                  604.6
PATB Pri LED X                   812.6
PATB Pri LED Y                   605.9
date               2023-04-01 00:00:00
Name: PATB, dtype: object


,X,Y,Z,Rx,Ry,z_sMATF,z_PATB,z_type,sMATF AC AZ,sMATF AC EL,PATB AC AZ,PATB AC EL,sMATF Pri LED X,sMATF Pri LED Y,PATB Pri LED X,PATB Pri LED Y,date
PATB,88.435,109.266,0,0.1018,0.0856,4027.174,3455.684,retro,0.0041,0.006,0.0041,0.0035,811.9,604.6,812.6,605.9,2023-04-01


### 1.1.2 Update the PATB pose to account for imperfect nulling.

In [6]:
#Apply a correction to account for imperfect nulling between PATB/sMATF and overwrite the values in the p_null_PATB_baseline_encoder DataFrame with the optimal ones.
optimize_PATB_for_imperfect_nulling = True

#Save a copy of the p_null_PATB dataframe, which we'll write to an Excel file later
p_null_PATB_baseline_encoder_original = p_null_PATB_baseline_encoder.copy()

if optimize_PATB_for_imperfect_nulling == True:
    p_null_PATB_baseline_encoder = optimize_p_null_PATB_encoders(p_null_PATB_baseline_encoder,p_null_offset)

Starting GSARX/RY:  0.1018 0.0856
Optimal GSARX/RY:  0.1032160175 0.08766031499999999
Starting HTSA/VTSA:  88.435 109.266
Optimal HTSA/VTSA:  88.446418 109.33648500000001


#### 1.1.5 Form a dataframe with the values supplied so far

In [7]:
#Generate a dataframe where we will store some parameters for each pose
columns = ['X','Y','Z','Rx','Ry']
df = pose_actual
df['color']='crimson'

gsa_rot = R.from_euler('x',GSA_angle_WCS_deg, degrees=True)
df.loc['sMask',['X','Y','Z']] = gsa_rot.apply(sMask)
df.loc['sMask','color'] = 'g'

dz_baseline = p_null_PATB_baseline_encoder.loc['PATB','z_sMATF'] - \
    p_null_PATB_baseline_encoder.loc['PATB','z_PATB']
df = pd.concat([df,p_null_PATB_baseline])
df.loc['PATB','color']='darkblue'

df.loc['sMPA',['X','Y','Z']] = sMPA_to_WCS.values.astype(float)
df.loc['sMPA','color']='purple'

for pose in df.index:
    update_uvec(df,pose,length=focal_length,rotangle=GSA_angle_WCS_deg)
    if 'PDI' in pose:
        df.loc[pose,'color'] = 'yellow'

# create normal vector to sMPA
vec = np.array([0,0,1])
rotmat = R.from_euler('XYZ',sMPA_angle_to_WCS_deg, degrees=True)
df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']] = rotmat.apply(vec)

#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PDI1,-109.089755,-4.477723,21.386613,-0.941514,10.804298,yellow,242.214916,264.860490,1241.274628
PDI2,111.429511,-3.828453,21.664246,-0.884323,-10.784784,yellow,-241.782625,263.638476,1241.619009
PDI3,39.795278,52.011229,11.044961,4.741583,-3.798262,yellow,-85.594929,142.863551,1281.345137
PDI4,-38.810833,51.903508,11.309534,4.728098,3.961031,yellow,89.257250,143.137547,1281.064648
PR1,0.943594,1.280695,-0.338386,0.031453,0.035137,crimson,0.792405,248.143393,1268.071794
PR2,-109.505749,-45.867074,22.370475,-4.911732,10.680266,crimson,239.466788,350.312004,1220.458522
PR3,111.309055,-45.373337,22.058458,-4.857077,-10.600321,crimson,-237.694885,349.239187,1221.112189
PR4,103.076801,59.765562,30.696044,5.307985,-10.311998,crimson,-231.300660,128.369032,1264.754223
PR5,-101.43058,59.40225,30.809096,5.269783,10.410043,crimson,233.475701,129.171851,1264.272746
sMask,1.3107,116.244531,587.154,NaN,NaN,g,NaN,NaN,NaN


#### 1.1.6 Calculate the center of the sMATF flat in 5DOF coordinate system (currently assumes PATB is on-axis and aligned with GSA)

In [8]:
start_coords = np.array(df.loc['PATB',['X','Y','Z']])

rx = df.loc['PATB','Rx']
ry = df.loc['PATB','Ry']
gsa_rot = R.from_euler('XY',[rx+GSA_angle_WCS_deg,ry], degrees=True)
uvec = np.array([0.,0.,dz_baseline]+p_null_offset)
rot_uvec = gsa_rot.apply(uvec) #np.dot(gsa_rot.as_matrix(),uvec)

sMATF_coords = start_coords+rot_uvec

df.loc['sMATF',['X','Y','Z']]=sMATF_coords
df.loc['sMATF',['Rx','Ry']]=df.loc['PATB',['Rx','Ry']]
df.loc['sMATF','color']='orange'
update_uvec(df,'sMATF',length=dz_baseline,rotangle=GSA_angle_WCS_deg)
#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PDI1,-109.089755,-4.477723,21.386613,-0.941514,10.804298,yellow,242.214916,264.860490,1241.274628
PDI2,111.429511,-3.828453,21.664246,-0.884323,-10.784784,yellow,-241.782625,263.638476,1241.619009
PDI3,39.795278,52.011229,11.044961,4.741583,-3.798262,yellow,-85.594929,142.863551,1281.345137
PDI4,-38.810833,51.903508,11.309534,4.728098,3.961031,yellow,89.257250,143.137547,1281.064648
PR1,0.943594,1.280695,-0.338386,0.031453,0.035137,crimson,0.792405,248.143393,1268.071794
PR2,-109.505749,-45.867074,22.370475,-4.911732,10.680266,crimson,239.466788,350.312004,1220.458522
PR3,111.309055,-45.373337,22.058458,-4.857077,-10.600321,crimson,-237.694885,349.239187,1221.112189
PR4,103.076801,59.765562,30.696044,5.307985,-10.311998,crimson,-231.300660,128.369032,1264.754223
PR5,-101.43058,59.40225,30.809096,5.269783,10.410043,crimson,233.475701,129.171851,1264.272746
sMask,1.3107,116.244531,587.154,NaN,NaN,g,NaN,NaN,NaN


## 1.2 ADM Update

Now the system has undergone some kind of change that will cause the ADM measurements to be different, such as shimming the sWFI or cooling from ambient to operational temps. We need to use the new set of ADM sMATF/PATB measurements to update the calculated poses for PRs 1-5.

Now update all poses from this new PATB pose; this is the recipe that would be followed for real data

#### 1.2.1 Input new ADM data for the sMATF and PATB and convert to 5DOF actual space

In [9]:
#p_null_PATB_update_encoder = pd.DataFrame(dict({'X':88.526, 'Y':108.364,'Z':0,
#                                                  'Rx':0.1287,'Ry':0.0788,
#                                                  'z_sMATF':4.031636*1000.,
#                                                  'z_PATB':3.458517*1000.,
#                                                  'z_type':'retro',
#                                                  'sMATF AC AZ':-0.0022,
#                                                  'sMATF AC EL':0.0042,
#                                                  'PATB AC AZ':-0.0008,
#                                                  'PATB AC EL':0.0001,
#                                                  'sMATF Pri LED X':811.7,
#                                                  'sMATF Pri LED Y':597.6,
#                                                  'PATB Pri LED X':812.1,
#                                                  'PATB Pri LED Y':598.8,
#                                                  'date':'4/3/2023'
#                                                 }),index=['PATB_update'])
p_null_PATB_update_encoder = get_data_from_ADM_log('SES WP2','retro',index_name='PATB_update',print_details=True) 
p_null_PATB_update = calculate_5DOF_from_encoders(p_null_PATB_update_encoder)
p_null_PATB_update

X                               88.526
Y                              108.364
Z                                    0
Rx                              0.1287
Ry                              0.0788
z_sMATF                       4031.636
z_PATB                        3458.517
z_type                           retro
sMATF AC AZ                    -0.0022
sMATF AC EL                     0.0042
PATB AC AZ                     -0.0008
PATB AC EL                      0.0001
sMATF Pri LED X                  811.7
sMATF Pri LED Y                  597.6
PATB Pri LED X                   812.1
PATB Pri LED Y                   598.8
date               2023-04-03 00:00:00
Name: PATB_update, dtype: object


,X,Y,Z,Rx,Ry
PATB_update,88.614997,108.375956,-0.149157,0.126749,0.078144


### 1.2.2 Update the PATB pose to account for imperfect nulling

In [10]:
#Apply a correction to account for imperfect nulling between PATB/sMATF and overwrite the values in the p_null_PATB_baseline_encoder DataFrame with the optimal ones.
optimize_PATB_for_imperfect_nulling = True

#Save a copy of the p_null_PATB dataframe, which we'll write to an Excel file later
p_null_PATB_update_encoder_original = p_null_PATB_update_encoder.copy()

if optimize_PATB_for_imperfect_nulling == True:
    p_null_PATB_update_encoder = optimize_p_null_PATB_encoders(p_null_PATB_update_encoder,p_null_offset)

Starting GSARX/RY:  0.1287 0.0788
Optimal GSARX/RY:  0.1298684923 0.0829718906
Starting HTSA/VTSA:  88.526 108.364
Optimal HTSA/VTSA:  88.54602399999999 108.421932


#### 1.2.2 Use the newly-provided data to calculate an updated position for sMATF

In [11]:
dz_update = p_null_PATB_update_encoder.loc['PATB_update','z_sMATF'] - \
    p_null_PATB_update_encoder.loc['PATB_update','z_PATB']
df = pd.concat([df,p_null_PATB_update])
df.loc['PATB_update','color'] = 'purple'
update_uvec(df,'PATB_update',length=focal_length,rotangle=GSA_angle_WCS_deg)

start_coords = np.array(df.loc['PATB_update',['X','Y','Z']])

rx = df.loc['PATB_update','Rx']
ry = df.loc['PATB_update','Ry']
gsa_rot = R.from_euler('XY',[rx+GSA_angle_WCS_deg,ry], degrees=True)
uvec = [0.,0.,dz_update]+p_null_offset # in GSA CS
rot_uvec = gsa_rot.apply(uvec) #np.dot(gsa_rot.as_matrix(),uvec)

sMATF_update_coords = start_coords+rot_uvec

df.loc['sMATF_update',['X','Y','Z']]=sMATF_update_coords
df.loc['sMATF_update',['Rx','Ry']]=df.loc['PATB_update',['Rx','Ry']]
df.loc['sMATF_update','color']='brown'
update_uvec(df,'sMATF_update',length=dz_update,rotangle=GSA_angle_WCS_deg)
#ax = plot_poses(df)
#plot_sMPA(df, ax)
print('Baseline poses')
df


Baseline poses


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PDI1,-109.089755,-4.477723,21.386613,-0.941514,10.804298,yellow,242.214916,264.860490,1241.274628
PDI2,111.429511,-3.828453,21.664246,-0.884323,-10.784784,yellow,-241.782625,263.638476,1241.619009
PDI3,39.795278,52.011229,11.044961,4.741583,-3.798262,yellow,-85.594929,142.863551,1281.345137
PDI4,-38.810833,51.903508,11.309534,4.728098,3.961031,yellow,89.257250,143.137547,1281.064648
PR1,0.943594,1.280695,-0.338386,0.031453,0.035137,crimson,0.792405,248.143393,1268.071794
PR2,-109.505749,-45.867074,22.370475,-4.911732,10.680266,crimson,239.466788,350.312004,1220.458522
PR3,111.309055,-45.373337,22.058458,-4.857077,-10.600321,crimson,-237.694885,349.239187,1221.112189
PR4,103.076801,59.765562,30.696044,5.307985,-10.311998,crimson,-231.300660,128.369032,1264.754223
PR5,-101.43058,59.40225,30.809096,5.269783,10.410043,crimson,233.475701,129.171851,1264.272746
sMask,1.3107,116.244531,587.154,NaN,NaN,g,NaN,NaN,NaN


In [12]:
#How did the distance between PATB and sMATF change from pre- to post-environment shift?
print('PATB->sMATF distance changed by: ',dz_update-dz_baseline, ' mm')

PATB->sMATF distance changed by:  1.63279206946072  mm


## 1.3 Calculate deltas between baseline & updated ADM measurements

In [13]:
sMATF_delta = df.loc['sMATF_update',['X','Y','Z']] - df.loc['sMATF',['X','Y','Z']]
print('sMATF (X,Y,Z) deltas: \n',sMATF_delta,'\n')

patb_delta = df.loc['PATB_update',['X','Y','Z','Rx','Ry']] - df.loc['PATB',['X','Y','Z','Rx','Ry']]
#patb_drxry = df.loc['PATB_update',['Rx','Ry']] - df.loc['PATB',['Rx','Ry']]
print('PATB (X,Y,Z,Rx,Ry) deltas: \n',patb_delta,'\n')


sMATF (X,Y,Z) deltas: 
 X    0.026919
Y   -0.862283
Z    1.592641
dtype: object 

PATB (X,Y,Z,Rx,Ry) deltas: 
 X     0.091025
Y    -0.902548
Z     0.000495
Rx    0.026894
Ry   -0.006767
dtype: object 



## 1.4 Transform old poses to the new sMATF reference frame

### 1.4.1 Apply a translation to all poses

In [14]:
df_update = df.copy()
T = df.loc['sMATF_update',['X','Y','Z']]
df_update[['X','Y','Z']] -= T
print('Translation required to put sMATF_update at the origin: \n',T)

Translation required to put sMATF_update at the origin: 
 X     -0.616888
Y     107.63641
Z    573.736708
Name: sMATF_update, dtype: object


### 1.4.2 Now perform 3 rotations to all the poses:
1. To align to the sMATF frame
2. To rotate everything by delta(Rx,Ry)
3. To undo #1

In [15]:
update_rot1 = R.from_euler('XY',[df.loc['PATB','Rx']+GSA_angle_WCS_deg,df.loc['PATB','Ry']], degrees=True)
update_rot2 = R.from_euler('XY',[patb_delta.loc['Rx'],patb_delta.loc['Ry']], degrees=True)
update_rot_combined = update_rot1 * update_rot2 * update_rot1.inv()
                                
for pose in df_update.index:
                                
    df_update.loc[pose,['X','Y','Z']]=update_rot_combined.apply(df_update.loc[pose,['X','Y','Z']].astype(float)) #np.dot(update_rot.as_matrix(),df_update.loc[pose,['X','Y','Z']])
    df_update.loc[pose,['uvec_X','uvec_Y','uvec_Z']] = update_rot_combined.apply(df_update.loc[pose,['uvec_X','uvec_Y','uvec_Z']].astype(float)) 
    if pose not in ['sMask','sMPA']:
        update_RxRy(df_update,pose,GSA_angle_WCS_deg)


### 1.4.3 Translate back to the 5DOF frame by undoing the translation from Cell 1.4.1, followed by adding the sMATF delta(X,Y,Z) offset from Cell 1.3

In [16]:
df_update[['X','Y','Z']] += T
df_update.loc[df_update.index[:12],['X','Y','Z']] += update_rot_combined.apply(sMATF_delta.astype(float)) #np.dot(update_rot.as_matrix(),delta)
#ax = plot_poses(df_update)
#plot_sMPA(df_update, ax)
print('Updated poses')
df_update

Updated poses


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PDI1,-108.996438,-5.083837,22.9137,-0.914589,10.797533,yellow,242.065047,264.283097,1241.426923
PDI2,111.522781,-4.429889,23.217236,-0.857458,-10.791549,yellow,-241.932504,263.050367,1241.714547
PDI3,39.888559,51.41321,12.615847,4.768511,-3.805011,yellow,-85.746780,142.260214,1281.402110
PDI4,-38.717579,51.303651,12.871245,4.754957,3.954283,yellow,89.105424,142.538155,1281.142047
PR1,1.039306,0.687177,1.204179,0.058347,0.028370,crimson,0.639792,247.548159,1268.188214
PR2,-109.411641,-46.473654,23.878086,-4.884719,10.673528,crimson,239.317467,349.744313,1220.650611
PR3,111.403187,-45.974956,23.591933,-4.830300,-10.607059,crimson,-237.844255,348.660783,1221.248384
PR4,103.167631,59.159698,32.277913,5.334983,-10.318741,crimson,-231.450267,127.770307,1264.787480
PR5,-101.339753,58.791874,32.367055,5.296573,10.403300,crimson,233.326129,128.583488,1264.360333
sMask,1.335708,115.375244,588.750501,NaN,NaN,g,NaN,NaN,NaN


# 1.4 >>>>Results<<<<

#### 1.4.1 Converting poses from real space to encoder space

In [17]:
df_update_encoder_space = convert_df_to_encoder_space(df_update)
print('Updated Poses (in encoder space): \n',df_update_encoder_space,'\n\n')

print('Baseline Poses (in encoder space), just for easy reference: \n',pose_encoders)

Updated Poses (in encoder space): 
                X          Y          Z        Rx         Ry
PDI1 -108.970707  -5.038501  22.984956 -0.906083  10.798478
PDI2  111.401957  -4.343440  23.372004 -0.862315 -10.789355
PDI3   39.822611  51.440843  12.747958  4.768260  -3.809335
PDI4  -38.731620  51.316989  12.974646  4.759410   3.950811
PR1     1.011409   0.723755   1.261070  0.060240   0.029097
PR2  -109.384878 -46.402928  23.927486 -4.877442  10.676486
PR3   111.283738 -45.863674  23.723266 -4.836208 -10.602907
PR4   103.041679  59.218034  32.491471  5.330563 -10.322534
PR5  -101.330136  58.812384  32.504309  5.305002  10.398571 


Baseline Poses (in encoder space), just for easy reference: 
                  X          Y          Z        Rx         Ry
Name.1                                                       
PDI1   -109.062609  -4.434582  21.454053 -0.933014  10.805269
PDI2    111.310127  -3.744227  21.815125 -0.889186 -10.782564
PDI3     39.730784  52.036620  11.173132  4.741350 

#### (OPTIONAL) 1.4.2 Display some other tables that may be helpful for diagnostics

In [18]:
print_extra_diagnostics = False
poses_to_display = [val for val in df.index if 'PDI' in val or 'PR' in val]+['sMask', 'sMPA']

if print_extra_diagnostics == True:
    print('Baseline Poses (in real space): \n',df.loc[poses_to_display,['X','Y','Z','Rx','Ry']],'\n\n')
    print('Updated Poses (in real space): \n',df_update.loc[poses_to_display,['X','Y','Z','Rx','Ry']], '\n\n')
    print('Deltas between Baseline/Updated poses (in real space): \n',-df.loc[poses_to_display,['X','Y','Z','Rx','Ry']] + df_update.loc[poses_to_display,['X','Y','Z','Rx','Ry']], '\n\n')
    print('Change in calculated endpoint positions (5DOF frame): \n',compute_endpoint_errors(df,df_update),'\n\n')
    print('Change in calculated endpoint positions (sMPA frame): \n',compute_endpoint_errors_sMPA_frame(df,df_update),'\n\n')
    print('Verify that all poses intersect at the pupil in the same way before and after the update:')
    print(check_pupil_crossing([df,df_update]),'\n\n')
    print('Distance from pose focus to sMPA: \n\n',compute_distance_to_sMPA(df),'\n\n')  
    

# 2 Write the new poses to an Excel file in the current directory

In [20]:
write_to_excel = True

if write_to_excel == True:
    write_new_poses_to_Excel('test output (from ADM update).xlsx','temporary plateau name',columns=columns,GSA_angle_WCS_deg=GSA_angle_WCS_deg,
                             df=df,df_encoders=pose_encoders,df_update=df_update,df_update_encoders=df_update_encoder_space,
                            focal_length=focal_length,p_null_PATB_baseline_encoder_original=p_null_PATB_baseline_encoder_original,
                            p_null_PATB_update_encoder_original=p_null_PATB_update_encoder_original)

**Writing to Excel complete**
